In [1]:
import numpy as np
import matplotlib.pyplot as plt
import MDSplus as mds
import os

os.environ['d3dpcs_path']='/fusion/projects/codes/pcs/data/ptdata/uncomp/mdsdata'
os.environ['d3d_path'] = 'atlas.gat.com::'

gyrotron_order = {
    'G1': 1,
    'G2': 2,
    'G3': 3,
    'G4': 4,
    'G5': 5,
    'G6': 6,
    'G7': 7,
    'G8': 8,
    'G9': 9,
    'G10': 10,
    'G11': 11
}

gyrotron_power_supply = {
    'G1': 0,
    'G2': 0,
    'G3': 0,
    'G4': 1,
    'G5': 2,
    'G6': 0,
    'G7': 0,
    'G8': 2,
    'G9': 4,
    'G10': 0,
    'G11': 3
}

power_per_gyrotron = {
    'G4': 0.51,
    'G5': 0.53,
    'G8': 0.58,
    'G9': 0.66,
    'G11': 0.56,
}

In [ ]:
#shotn = 205820
shotn = 205834

connection = mds.Connection('atlas.gat.com')
connection.openTree('d3d', shotn)

target_times = connection.get(f'dim_of(PTDATA(\'EOXTARGET\', {shotn}))')
target = connection.get(f'PTDATA(\'EOXTARGET\', {shotn})')
best_times = connection.get(f'dim_of(PTDATA(\'EOXBEST\', {shotn}))')
best = connection.get(f'PTDATA(\'EOXBEST\', {shotn})')
# Reshape data
best_reshaped = best.value.reshape(-1, 11, 101).copy()
best_times_reshaped = best_times.value[:best_reshaped.shape[0]]
target_reshaped = target.value.reshape(-1, 1, 101)
target_times_reshaped = target_times.value[:target_reshaped.shape[0]]

best_reshaped_reordered = np.copy(best_reshaped)
for gt_name, gt_index in gyrotron_order.items():
    # Extract original gyrotron number from name (e.g., 'G8' -> 8)
    original_index = int(gt_name[1:]) - 1  # 0-indexed position in original array
    new_index = gt_index - 1  # 0-indexed position in reordered array
    best_reshaped_reordered[:, new_index, :] = best_reshaped[:, original_index, :]

scaled_power_per_gyrotron = {}
for power_supply in range(1,6):
    gyrotrons_to_alter = [gt for gt, ps in gyrotron_power_supply.items() if ps == power_supply]
    dc = connection.get(f'PTDATA(\'EOSBDC{power_supply}\', {shotn})')
    dc_times = connection.get(f'dim_of(PTDATA(\'EOSBDC{power_supply}\', {shotn}))')
    # downsample dc to match best_times_reshaped
    dc_downsampled = np.interp(best_times_reshaped, dc_times, dc)
    for gt in gyrotrons_to_alter:
        scaled_power_per_gyrotron[f"{gt}"] = power_per_gyrotron[gt] * dc_downsampled


# faulting logic
did_it_fault_downsampled = {}
for gt_number in [4,5,8,9,11]:
    faults = connection.get(f'PTDATA(\'GYSFDURG{gt_number}\', {shotn})')
    faults_time = connection.get(f'dim_of(PTDATA(\'GYSFDURG{gt_number}\', {shotn}))')
    # a fault is if faults value > 0.01
    did_it_fault = faults > 0.1
    # interpolate to best_times_reshaped
    did_it_fault_downsampled[f'G{gt_number}'] = np.interp(best_times_reshaped, faults_time, did_it_fault.astype(float))

for time_ind, time in enumerate(best_times_reshaped):
    best_reshaped_reordered[time_ind, 3, :] = best_reshaped_reordered[time_ind, 3, :]*scaled_power_per_gyrotron['G4'][time_ind]*(1-did_it_fault_downsampled['G4'][time_ind]) # G4
    best_reshaped_reordered[time_ind, 4, :] = best_reshaped_reordered[time_ind, 4, :]*scaled_power_per_gyrotron['G5'][time_ind]*(1-did_it_fault_downsampled['G5'][time_ind])  # G5
    best_reshaped_reordered[time_ind, 7, :] = best_reshaped_reordered[time_ind, 7, :]*scaled_power_per_gyrotron['G8'][time_ind]*(1-did_it_fault_downsampled['G8'][time_ind])  # G8
    best_reshaped_reordered[time_ind, 8, :] = best_reshaped_reordered[time_ind, 8, :]*scaled_power_per_gyrotron['G9'][time_ind]*(1-did_it_fault_downsampled['G9'][time_ind])  # G9
    best_reshaped_reordered[time_ind, 10, :] = best_reshaped_reordered[time_ind, 10, :]*scaled_power_per_gyrotron['G11'][time_ind]*(1-did_it_fault_downsampled['G11'][time_ind])  # G11

from ipywidgets import interact, IntSlider
import matplotlib.pyplot as plt
import numpy as np

best_summed = best_reshaped_reordered.sum(axis=1)  # Shape: (time_steps, 101)

# Get the actual number of time steps from the data
n_time_steps = best_summed.shape[0]

# Gyrotrons to plot
gyros_to_plot = [4, 5, 8, 9, 11]

# Create rho axis from 0 to 1
rho = np.linspace(0, 1, 101)

def plot_profiles(time_index):
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(8, 6))
    
    # Top subplot: Summed best and target
    ax1.plot(rho, best_summed[time_index, :], label='Best (Sum of 11 Gyrotrons)', linewidth=2)
    ax1.plot(rho, target_reshaped[time_index, 0, :], label='Target', linewidth=2, linestyle='--')
    ax1.set_xlabel('rho')
    ax1.set_ylabel('Power Density')
    if time_index < len(best_times):
        ax1.set_title(f'ECH Profiles at Time Index {time_index} (t = {float(best_times[time_index]):.3f} s)')
    else:
        ax1.set_title(f'ECH Profiles at Time Index {time_index}')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # Bottom subplot: Individual gyrotrons (only G4, G5, G8, G9, G11)
    for gyro_num in gyros_to_plot:
        ax2.plot(rho, best_reshaped_reordered[time_index, gyro_num - 1, :], label=f'G{gyro_num}', alpha=0.7, linewidth=2)
    ax2.set_xlabel('rho')
    ax2.set_ylabel('Power Density')
    ax2.set_title('Individual Gyrotrons (G4, G5, G8, G9, G11)')
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show(3)

# Create interactive slider using actual data dimensions
interact(plot_profiles, time_index=IntSlider(min=0, max=n_time_steps-1, step=1, value=min(50, n_time_steps-1), description='Time Index'))

interactive(children=(IntSlider(value=50, description='Time Index', max=99), Output()), _dom_classes=('widget-…

<function __main__.plot_profiles(time_index)>

In [5]:
# Create animated GIF of ECH profiles over time
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation, PillowWriter
import numpy as np

# Create the figure and axes
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(8, 6))

def animate(time_index):
    # Clear both axes
    ax1.clear()
    ax2.clear()
    
    # Top subplot: Summed best and target
    ax1.plot(rho, best_summed[time_index, :], label='Best (Sum of 11 Gyrotrons)', linewidth=2)
    ax1.plot(rho, target_reshaped[time_index, 0, :], label='Target', linewidth=2, linestyle='--')
    ax1.set_xlabel('rho')
    ax1.set_ylabel('Power Density')
    if time_index < len(best_times):
        ax1.set_title(f'ECH Profiles at Time Index {time_index} (t = {float(best_times[time_index]):.3f} s)')
    else:
        ax1.set_title(f'ECH Profiles at Time Index {time_index}')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # Bottom subplot: Individual gyrotrons (only G4, G5, G8, G9, G11)
    for gyro_num in gyros_to_plot:
        ax2.plot(rho, best_reshaped_reordered[time_index, gyro_num - 1, :], label=f'G{gyro_num}', alpha=0.7, linewidth=2)
    ax2.set_xlabel('rho')
    ax2.set_ylabel('Power Density')
    ax2.set_title('Individual Gyrotrons (G4, G5, G8, G9, G11)')
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()

# Create animation with every timestep starting from index 42
frame_step = 1
frames = range(42, n_time_steps, frame_step)

anim = FuncAnimation(fig, animate, frames=frames, interval=500, repeat=True)

# Save as GIF
output_file = 'ech_profiles_animation.gif'
writer = PillowWriter(fps=2)
anim.save(output_file, writer=writer)
print(f"Animation saved to {output_file}")

plt.close(fig)

Animation saved to ech_profiles_animation.gif
